### Установка зависимостей


In [ ]:
!apt-get install xvfb
!pip install gym[atari] autorom[accept-rom-license] gymnasium[mujoco] stable-baselines3 PyVirtualDisplay --quiet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.7).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install -q swig
!pip install -q gymnasium

In [ ]:
!pip install -q 'shimmy>=0.2.1'

In [ ]:
# Импорт необходимых пакетов
import gymnasium as gym
from stable_baselines3 import DQN

### Выбор среды
Давайте определим задачу, которую мы будем решать, используя алгоритм Q-learning и библиотеки gym и stable-baselines3.


#### Создание среды Taxi-v3

Мы используем метод gym.make() для создания среды с именем 'LunarLander-v2'.

In [ ]:
ENV_NAME = "Taxi-v3"
TIMESTEPS = 400_000

In [ ]:

env = gym.make(ENV_NAME)

#### Описание задачи
Мы сохраняем информацию о пространстве наблюдений (observation_space) и пространстве действий (action_space). observation_space представляет собой пространство всех возможных состояний в задаче, а action_space - пространство всех возможных действий, которые агент может предпринять.

In [ ]:
# Описание задачи
observation_space = env.observation_space
action_space = env.action_space

Эти шаги предварительно готовят нас к использованию выбранной среды для обучения и тестирования агента с использованием алгоритма Q-learning.

### Решение задачи
На этом шаге мы используем алгоритм Q-learning, представленный в stable-baselines3, для решения задачи CartPole. Давайте разберем каждый этап.


#### Инициализация модели DQN
Мы создаем экземпляр модели Deep Q-Network (DQN) с использованием DQN("MlpPolicy", env, verbose=1). Здесь "MlpPolicy" означает полносвязную нейронную сеть в качестве политики. env - это среда CartPole, а verbose=1 добавляет вывод для отслеживания прогресса обучения.

In [ ]:
# Инициализация модели DQN
model = DQN("MlpPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


#### Обучение модели
Мы вызываем метод learn() для обучения модели на 400000 временных шагах (total_timesteps=400000).

In [ ]:
# Обучение модели
model.learn(total_timesteps=TIMESTEPS, log_interval=1000)
model.save("dqn_lopatin_lr1" )

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -227     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1000     |
|    fps              | 832      |
|    time_elapsed     | 239      |
|    total_timesteps  | 199141   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000512 |
|    n_updates        | 37285    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -224     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2000     |
|    fps              | 745      |
|    time_elapsed     | 535      |
|    total_timesteps  | 399141   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000276 |
|    n_updates      

#### Тестирование модели
Мы тестируем обученную модель, используя цикл, в котором агент принимает решения в соответствии с обученной политикой на каждом временном шаге. Мы используем метод render(), чтобы визуализировать среду в реальном времени.

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
import gym
from tqdm import tqdm
from IPython import display
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
from matplotlib import animation


d = Display()
d.start()

env = gym.make(ENV_NAME)

obs = env.reset()

img = []
for _ in tqdm(range(1000)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated = env.step(int(action))

    display.clear_output(wait=True)
    img.append(env.render('rgb_array'))

dpi = 72
interval = 50 # ms

plt.figure(figsize=(img[0].shape[1]/dpi,img[0].shape[0]/dpi),dpi=dpi)
patch = plt.imshow(img[0])
plt.axis=('off')
animate = lambda i: patch.set_data(img[i])
ani = animation.FuncAnimation(plt.gcf(),animate,frames=len(img),interval=interval)
display.display(display.HTML(ani.to_jshtml()))


100%|██████████| 1000/1000 [00:23<00:00, 42.43it/s]


#### Закрытие среды после тестирования
По завершении тестирования мы закрываем среду с помощью env.close().

In [ ]:
# Закрытие среды после тестирования
env.close()